## Library

In [70]:
import cv2
import os
import datetime
import numpy as np
import pandas as pd
from pathlib import Path

## File Path

In [71]:
# File Path
IMAGE_DIR = r"D:\Code\Python\Jupyter\OpenCV Image Record\Capture_Image"
EXCEL_DIR = r"D:\Code\Python\Jupyter\OpenCV Image Record\Excel_Records"
CASCADE_PATH = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"

# File Name
EXCEL_FILE = "attendance_record.xlsx"

## Create a Directory for Storing File

In [72]:
def create_directories():
    """Create necessary directories if they don't exist."""
    directories = [IMAGE_DIR, EXCEL_DIR]
    for directory in directories:
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(f"Created directory: {directory}")
def create_save_directory(path):
    """Create the image save directory if it doesn't exist."""
    if not os.path.exists(path):
        os.makedirs(path)

## Create Excel

In [73]:
def initialize_excel_file():
    """Initialize Excel file with headers if it doesn't exist."""
    excel_path = os.path.join(EXCEL_DIR, EXCEL_FILE)
    
    if not os.path.exists(excel_path):
        # Create new Excel file with headers
        df = pd.DataFrame(columns=['Date', 'Time', 'Image_Filename', 'Image_Path'])
        df.to_excel(excel_path, index=False)
        print(f"Created new Excel file: {excel_path}")
    return excel_path

def add_to_excel(image_filename, image_path):
    """Add a new attendance record to the Excel file."""
    excel_path = os.path.join(EXCEL_DIR, EXCEL_FILE)
    
    try:
        # Read existing data
        df = pd.read_excel(excel_path)
        
        # Get current timestamp
        now = datetime.datetime.now()
        
        # Create new record
        new_record = {
            'Date': now.strftime("%Y-%m-%d"),
            'Time': now.strftime("%H:%M:%S"),
            'Image_Filename': image_filename,
            'Image_Path': image_path,
        }
        
        # Add new record to dataframe
        new_df = pd.DataFrame([new_record])
        df = pd.concat([df, new_df], ignore_index=True)
        
        # Save back to Excel
        df.to_excel(excel_path, index=False)
        print(f"Added record to Excel: {image_filename}")
        
    except Exception as e:
        print(f"Error adding to Excel: {str(e)}")

In [74]:
def initialize_camera(index=0):
    """Initialize the webcam."""
    cap = cv2.VideoCapture(index)
    if not cap.isOpened():
        raise IOError("Error: Cannot open the camera.")
    return cap

def load_face_detector():
    """Load Haar cascade for face detection."""
    return cv2.CascadeClassifier(CASCADE_PATH)

def create_alignment_box(size=(200, 250)):
    """Create a simple alignment box instead of loading an avatar."""
    # No need to load any image file - we'll draw the box directly
    return size

def draw_alignment_box(frame, position, size, color=(0, 255, 0), thickness=3):
    """Draw a simple alignment box on the frame."""
    x, y = position
    w, h = size
    
    cv2.rectangle(frame, (x, y), (x + w, y + h), color, thickness)
    
    # Draw corner markers for better visibility
    corner_length = 40
    corner_thickness = 4
    corner_color = (255, 255, 0)  # Yellow corners
    
    # Top-left corner
    cv2.line(frame, (x, y), (x + corner_length, y), corner_color, corner_thickness)
    cv2.line(frame, (x, y), (x, y + corner_length), corner_color, corner_thickness)
    
    # Top-right corner
    cv2.line(frame, (x + w, y), (x + w - corner_length, y), corner_color, corner_thickness)
    cv2.line(frame, (x + w, y), (x + w, y + corner_length), corner_color, corner_thickness)
    
    # Bottom-left corner
    cv2.line(frame, (x, y + h), (x + corner_length, y + h), corner_color, corner_thickness)
    cv2.line(frame, (x, y + h), (x, y + h - corner_length), corner_color, corner_thickness)
    
    # Bottom-right corner
    cv2.line(frame, (x + w, y + h), (x + w - corner_length, y + h), corner_color, corner_thickness)
    cv2.line(frame, (x + w, y + h), (x + w, y + h - corner_length), corner_color, corner_thickness)
    
    # Add center cross for alignment reference
    center_x, center_y = x + w//2, y + h//2
    cross_size = 30
    cross_color = (255, 0, 255)  # Magenta cross
    cv2.line(frame, (center_x - cross_size, center_y), (center_x + cross_size, center_y), cross_color, 3)
    cv2.line(frame, (center_x, center_y - cross_size), (center_x, center_y + cross_size), cross_color, 3)
    
    return frame

def is_face_aligned(face, box_pos, box_size, tolerance=0.25):
    """Check if face is properly aligned with the alignment box."""
    fx, fy, fw, fh = face
    bx, by, bw, bh = box_pos + box_size
    
    # Calculate face center and box center
    face_center_x = fx + fw // 2
    face_center_y = fy + fh // 2
    box_center_x = bx + bw // 2
    box_center_y = by + bh // 2
    
    # Calculate distance from face center to box center
    distance_x = abs(face_center_x - box_center_x)
    distance_y = abs(face_center_y - box_center_y)
    
    # Check if face is within tolerance range
    max_distance_x = bw * tolerance
    max_distance_y = bh * tolerance
    
    # Also check if face size is reasonable compared to box
    size_ratio = (fw * fh) / (bw * bh)
    size_ok = 0.1 < size_ratio < 0.95  # Face should be 15%-85% of box area
    
    return (distance_x < max_distance_x and distance_y < max_distance_y and size_ok)

In [75]:
def main():

    create_directories()
    excel_path = initialize_excel_file()

    create_save_directory(IMAGE_DIR)
    cap = initialize_camera()
    face_cascade = load_face_detector()

    # Create alignment box
    BOX_SIZE = create_alignment_box()

    print("Align your face with the avatar outline. Press SPACE to capture.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(100, 100))     # Haar Cascades Equation

        # Calculate avatar position (lower center)
        height, width = frame.shape[:2]
        BOX_POS = ((width - BOX_SIZE[0]) // 2, (height - BOX_SIZE[1]) // 2)

        # Process faces (only keep largest if multiple exist)
        main_face = None
        if len(faces) > 0:
            main_face = max(faces, key=lambda x: x[2]*x[3])  # Largest face     Area = width × height.
            faces = [main_face]  # Only track one face

        # Prepare display frame
        display_frame = frame.copy()
        display_frame = draw_alignment_box(display_frame, BOX_POS, BOX_SIZE)

        # Face detection and alignment feedback
        alignment_status = "No face detected"
        is_aligned = False
        
        if main_face is not None:
            x, y, w, h = main_face
            is_aligned = is_face_aligned(main_face, BOX_POS, BOX_SIZE)

            if is_aligned:
                color = (0, 255, 0)  # Green
                alignment_status = "Aligned - Press SPACE to capture"
                thickness = 2
            else:
                color = (0, 0, 255)  # Red  
                alignment_status = "Not aligned - Move your face"
                thickness = 2

            # Draw rectangle around face
            cv2.rectangle(display_frame, (x, y), (x + w, y + h), color, thickness)

        # UI Elements
        cv2.putText(display_frame, alignment_status, (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        cv2.putText(display_frame, "ESC: Exit | SPACE: Capture", (10, height-15), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

        cv2.imshow("Face Alignment Attendance System", display_frame)

        key = cv2.waitKey(1)
        if key == 27:  # ESC
            break
        elif key == 32:  # SPACE
            if main_face is not None and is_aligned:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                image_filename = f"aligned_face_{timestamp}.jpg"
                image_path = os.path.join(IMAGE_DIR, image_filename)
                
                cv2.imwrite(image_path, frame)
                print(f"Saved aligned face image")

                add_to_excel(image_filename, image_path)
            else:
                print("Capture failed: Face not aligned with avatar")

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Align your face with the avatar outline. Press SPACE to capture.
Saved aligned face image
Added record to Excel: aligned_face_20250623_154756.jpg
Capture failed: Face not aligned with avatar
Saved aligned face image
Added record to Excel: aligned_face_20250623_154800.jpg
